# PISM Greenland Tutorial

The flow of glaciers can be described as Stokes flow, obeying conservation equations for *mass*, *linear momentum*, and *energy* (*angular momentum* results in a symmetric stress tensor)

$$
\begin{aligned}
\textrm{mass}:  &\nabla \cdot \mathbf{v} & = & 0 \\
\textrm{momentum}:  & \nabla \left( \eta \nabla \cdot \mathbf{v} \right) - \rho \mathbf{g} & = & 0 \\
\textrm{energ}: & \frac{\mathrm{d} E}{\mathrm{d} t}
\end{aligned}
$$

The visosity of ice, $\eta = A(T) \left(v \right)$. Ice has a memory, which significantly complicates the simulation of glaciers.

We need initial conditions for these conservation equations.

mass: initial ice geometry $\Omega_0$
momentum: 
energy: initial energy field $E_0$
$$
\begin{aligned}
\textrm{mass}:  & \textrm{ice geometry} \Omega_0 \\
\textrm{momentum}: & \textrm{}  \\
\textrm{energy}: & \textrm{energy} E_0
\end{aligned}
$$

In [ ]:
import matplotlib.pylab as plt
from matplotlib import colors
import numpy as np
import xarray as xr

from pism_tutorials.plotting import register_colormap

register_colormaps()

In [ ]:
bucket_name = "pism-cloud-data"

In [ ]:
tutorial_files = {"initial_state": {"path": "initial_states", "files": ["g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc"]}, 
                  "basal_heatflux": {"path": "bheatflux", "files": ["geothermal_heat_flow_map_10km.nc"]},
                 "grid": {"path": "grids", "files": ["pism-bedmachine.nc"]}}

to_download = []
for forcing_type, forcing in tutorial_files.items():
    for f in forcing["files"]:
        d = Path(forcing["path"])
        d.mkdir(parents=True, exist_ok=True)
        p = d / Path(f)
        to_download.append(str(p))


In [ ]:
download_files(bucket_name, to_download, overwrite=False)

In [ ]:
print(to_download)

# s3 = boto3.resource('s3')

# my_bucket = s3.Bucket(bucket_name)

# for file in my_bucket.objects.all():
#     print(file.key)

s3_client=boto3.client('s3')
s3_client.download_file(bucket_name, 'initial_states/g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc', 'g1200m_id_BAYES-MEDIAN_1980-1-1_1984-12-31.nc')

In [ ]:
init_file = Path(tutorial_files["initial_state"]["path"]) / Path(tutorial_files["initial_state"]["files"][0])
init_ds = xr.open_dataset(init_file, decode_timedelta=True)

bhf_file = Path(tutorial_files["basal_heatflux"]["path"]) / Path(tutorial_files["basal_heatflux"]["files"][0])
bhf_ds = xr.open_dataset(bhf_file, decode_timedelta=True)

## Initial geometry

In [ ]:
fig, axs = plt.subplots(1, 3, sharex=True, sharey=True, figsize=(12, 4))
init_ds["usurf"].plot(ax=axs[0])
init_ds["topg"].plot(ax=axs[1])
init_ds["thk"].plot(ax=axs[2])
#bhf_ds["bheatflx"].plot(ax=axs[1,0], cmap="magma_r", vmin=40, vmax=80)
#init_ds["velsurf_mag"].plot(ax=axs[1,1], cmap=speed_cmap, vmin=10, vmax=1500)

In [ ]:
f = "mpirun -np 4 pism -eisII A -grid.Mx 101 -grid.My 101"

In [ ]:
!$f 